# python-quickstart

This page is a brisk introduction to Spell's [Python API](https://spell.run/docs/python). The Spell Python library provides programmatic access to the following APIs (click to go to the corresponding section of this guide):

* [Runs](#runs)
* [Resources](#resources)
* [Hyperparameter Searches](#hyperparameter-searches)
* [Models](#models)
* [Model Servers](#model-servers)

----

## runs

### run basics

First use `spell.client.from_environment` to instantiate the Spell Python client with your credentials.

This works inside of a Spell workspace or run automatically (your credentials are included in the run environment).

If you are on your local machine, note that you will first need to log in using the `spell login` CLI command. You only need to do this once.

In [1]:
import spell.client
client = spell.client.from_environment()

The Python equivalent to the `spell run` CLI command is `client.runs.new`. This function contains all of the same parameters (well, mostly) and returns a `Run` object you can use for further interaction with this run.

In [2]:
client.runs.new?

Signature: client.runs.new(**kwargs)
Docstring:
Create a run.

Args:
    command (str): the command to run
    machine_type (str, optional): the machine type for the run (default: CPU)
    project (str, optional): the project to associate this run with (default: None)
    workspace_id (int, optional): the workspace ID for code to include in the run (default: None)
    commit_hash (str, optional): a specific commit hash in the workspace corresponding to :obj:`workspace_id`
        for code to include in the run (default: None)
    commit_label (str, optional): a commit label for code to include in the run. Only applicable
        if this is a workflow run (i.e., the :py:attr:`~spell.client.SpellClient.active_workflow` of the
        client is set or a :obj:`workflow_id` is provided) (default: None). The value must correspond
        to one of the commit labels specified upon workflow creation using the ``--repo`` or ``--github-repo``
        options. Only applicable if a workspace is sp

In [2]:
r1 = client.runs.new(command="echo Hello World!")

In [3]:
r1

Run(id=355, status='machine_requested', command='echo Hello World!', creator=User(email='aleksey@spell.run', user_name='aleksey', full_name='Aleksey Bilogur', created_at=datetime.datetime(2020, 2, 12, 23, 28, 32, 771444, tzinfo=tzutc()), updated_at=datetime.datetime(2020, 4, 28, 0, 27, 17, 514510, tzinfo=tzutc()), last_logged_in=datetime.datetime(2020, 4, 28, 0, 27, 17, 390465, tzinfo=tzutc())), gpu='CPU', framework='default', created_at=datetime.datetime(2020, 5, 6, 15, 14, 29, 659725, tzinfo=tzutc()))

Each of these fields is an attribute on the run.

In [6]:
r1.id

355

In [8]:
r1.status

'machine_requested'

One of the most important `Run` attributes is the `status`. The `status` field reflects the state the run was in at the time the `Run` object was last generated or refreshed.

`client.runs.new` always exits as soon as the run is successfully queued, hence it always returns a `Run` object in the `'machine_requested'` state. To update the `status` field, run `refresh`.

In [9]:
r1.refresh()

In [10]:
r1.status

'complete'

You can also initialize a `Run` object from an existing run by ID.

In [23]:
r1 = client.runs.get(355)

### waiting on run completion

To wait until the run reaches a certain state, use `wait_status`. For a full list of states that a run can be in refer to ["Run States"](https://spell.run/docs/run_overview#advanced-run-states) in the runs documentation. In most cases you will simply want to wait until the run terminates (reaching one of the final states); you can then check for `status == 'complete'` to determine whether or not the run succeeded.

In [11]:
r2 = client.runs.new(command="echo Hello World Again!")
r2.wait_status(*client.runs.FINAL)
r2.refresh()
if r2.status == client.runs.COMPLETE:
    print("Run succeeded!")
else:
    print("Run failed!")

Run succeeded!


### getting run logs

The logs associated with a run are available as a sequence of `LogEntry` objects via `logs`. The `LogEntry` object provides the same information that the logs in the web console provide in a more Pythonic way.

In [16]:
list(r2.logs())

[LogEntry(status='machine_requested', log='Run created -- waiting for a CPU machine.', status_event=True, level='info', timestamp='2020-05-06T15:56:07+00:00'),
 LogEntry(status='building', log='Run is building', status_event=True, level='info', timestamp='2020-05-06T15:56:08+00:00'),
 LogEntry(status='building', log='Machine acquired -- commencing run', level='info', timestamp='2020-05-06T15:56:08+00:00'),
 LogEntry(status='building', log='Retrieving cached environment...', timestamp='2020-05-06T15:56:13+00:00'),
 LogEntry(status='running', log='Run is running', status_event=True, level='info', timestamp='2020-05-06T15:56:19+00:00'),
 LogEntry(status='running', log='Hello World Again!', timestamp='2020-05-06T15:56:21+00:00'),
 LogEntry(status='saving', log='Run is saving', status_event=True, level='info', timestamp='2020-05-06T15:56:22+00:00'),
 LogEntry(status='pushing', log='Run is pushing', status_event=True, level='info', timestamp='2020-05-06T15:56:26+00:00'),
 LogEntry(status='pu

To get only the log lines corresponding with user output, filter on `status == 'running'`.

In [18]:
[log for log in r2.logs() if log.status == client.runs.RUNNING]

[LogEntry(status='running', log='Run is running', status_event=True, level='info', timestamp='2020-05-06T15:56:19+00:00'),
 LogEntry(status='running', log='Hello World Again!', timestamp='2020-05-06T15:56:21+00:00')]

### terminating a run

You can `stop` or `kill` a run using the Python client in much the same way you can using the Spell CLI.

In [24]:
r3 = client.runs.new(command="sleep 1000")
r3.wait_status(client.runs.RUNNING)
r3.kill()  # or r3.stop()
r3.wait_status(*client.runs.FINAL)
r3.refresh()
r3.status

'killed'

To learn more about the difference between `stop` and `kill` refer to ["Interrupting a run"](https://spell.run/docs/run_overview#interrupting-a-run) in the Runs documentation.

### copying run resources

You can copy run resources to local disk using `cp`.

In [30]:
r4 = client.runs.new(command="echo Hello World > hello_world.txt")

In [38]:
r4.cp("hello_world.txt")

In [39]:
%ls

hello_world.txt   quickstart.ipynb


### working with run metrics

As the [Metrics](https://spell.run/docs/metrics/) page in the docs explains, you can use the `send_metric` method inside of a run to report model metrics to Spell, then use `run.metrics` to retrieve them.

In [49]:
# %load ../metrics/basic.py
import spell.metrics as metrics
import time
import argparse

# Runs for --steps seconds and sends --steps spell metrics with the key 'value'
# and a numeric value starting at --start and incrementing by --stepsize
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--start", type=float, help="Value to start at")
    parser.add_argument("--steps", type=int, help="Number of metrics to send")
    parser.add_argument("--stepsize", type=float, help="Size of step to take")
    args = parser.parse_args()

    value = args.start
    for i in range(args.steps):
        print("Sending metric {}".format(value))
        metrics.send_metric("value", value) 
        value += args.stepsize
        time.sleep(1)


In [54]:
r5 = client.runs.new(
    github_url="https://github.com/spellrun/examples.git", 
    command="python metrics/basic.py --start 1 --steps 4 --stepsize 1",
)
r5.wait_status(*client.runs.FINAL)

In [56]:
# %load ../metrics/read.py
import pandas as pd
import spell.client

client = spell.client.from_environment()

# replace with the actual run id value
RUN_ID = r5.id
run = client.runs.get(RUN_ID)

# we return the metrics data as a generator
metric = run.metrics("value")

df = pd.DataFrame(metric, columns=["timestamp", "index", "value"])
df

,timestamp,index,value
0,2020-05-06 18:20:50.544695+00:00,0,1
1,2020-05-06 18:20:51.550610+00:00,1,2
2,2020-05-06 18:20:52.554921+00:00,2,3
3,2020-05-06 18:20:53.559158+00:00,3,4


### assigning a run to a project

You can group your runs (and build dashboards for them) using [project](https://spell.ml/docs/project_overview/). To attach a new run to a project, using the `project` field:

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
r6 = client.runs.new(
    command="echo Hello World!",
    project="MNIST"
)
r6.wait_status(*client.runs.FINAL)

In [4]:
r6.project

Project(id=6, name='MNIST', description='Attempts to categorize images of hand drawn digits')

For more `run` methods refer to the [run docs](https://spell.ml/docs/runs/).

----

### working with workflows

The master run script in a workflow leverages the Python API to do its work. Refer to the `workflows` folder in this repository to learn more.

Note that it is not currently possible to launch a new workflow from the Python API.

----

## resources

You can interact with Spell resources via the Python API. The resources client has two methods: `ls` for viewing resources (`uploads/` for uploads, `runs/` for runs) and `cp` for downloading them to local disk.

In [1]:
import spell.client
client = spell.client.from_environment()

In [3]:
list(client.resources.ls("uploads/"))

[LsLine(path='100MDataset', date='2019-10-21T15:27:33.707682Z'),
 LsLine(path='cifar10', date='2020-08-11T00:10:35.147822Z'),
 LsLine(path='coconet-model', date='2020-11-10T18:47:37.711716Z'),
 LsLine(path='dogs', date='2020-03-11T15:13:02.542201Z'),
 LsLine(path='examples', date='2019-10-21T15:35:34.379891Z'),
 LsLine(path='fluent-logger-golang', date='2019-10-21T15:16:45.234666Z'),
 LsLine(path='guitar_chords', date='2020-03-15T17:22:43.22915Z'),
 LsLine(path='hptwo', date='2019-10-31T22:37:08.514779Z'),
 LsLine(path='JSB-Chorales-dataset', date='2020-11-10T17:28:10.325366Z'),
 LsLine(path='liberty-mutual-group-property-inspection-prediction', date='2020-08-10T23:08:31.005482Z'),
 LsLine(path='neural-style-imgs', date='2021-03-12T21:40:57.339617Z'),
 LsLine(path='output', date='2019-10-21T15:26:05.716872Z'),
 LsLine(path='paint_with_ml', date='2020-09-15T15:38:37.959968Z'),
 LsLine(path='pytorch-examples', date='2019-10-21T15:54:12.092107Z'),
 LsLine(path='serving', date='2021-01-26T

In [15]:
list(client.resources.ls("runs/1011"))

[LsLine(path='GuaGAN_Bob_Ross_From_ADE20K_Landscapes_No_VAE/'),
 LsLine(path='latest_net_G.yaml', size=612, date='2020-09-15T15:47:11Z')]

In [23]:
client.resources.cp("runs/17/keras/saved_models/", "checkpoints/")

Extracting keras_cifar10_trained_model.h5...


## hyperparameter searches

To launch a hyperparameter search using the Spell Python library, use one of the three launcher methods (`new_grid_search`, `new_random_search`, and `new_bayesian_search` in the `client.hyper.*` namespace.

To learn more about the hyperparameter search feature, refer to our blog post ["An introduction to hyperparameter search with CIFAR10"](https://spell.run/blog/an-introduction-to-hyperparameter-search-with-cifar10-Xo8_6BMAACEAkwVs), the hyperparameter search tutorial in the `hyper` folder in this repo, and/or our [hyperparameter search docs](https://spell.run/docs/hyper_searches) page.

The following code sample demonstrates this in action. Note that running this code cell launches a large hyperparameter search job, so be prepared for this if you do!

In [94]:
from spell.api.models import ValueSpec

h1 = client.hyper.new_grid_search(
    params={
        'conv2_filter': ValueSpec([16, 72, 128]),
        'dense_layer': ValueSpec([32, 64, 128]),
        'dropout_3': ValueSpec([0.2, 0.5])
    },
    command="python hyper/cifar10_cnn.py --epochs 25 --conv2_filter :conv2_filter: --dense_layer :dense_layer: --dropout_3 :dropout_3:",
    machine_type="K80",
    github_url="https://github.com/spellrun/examples.git",
)

The hyperparameter search methods accept all of the same parameters that the run creation method (`spell.client.new`) accepts, plus one new one, `params`, specifying the hyperparameter search space.

Note also that you will need to provide the run instruction explicitly via the `command` keyword argument.

The input to `params` can be a `dict` of `ValueSpec` objects wrapping a list, in the case of `new_grid_search`, or a `dict` or `RangeSpec` objects, in the case of `new_random_search` and `new_bayesian_search`. Here's an example of a random search using `RangeSpec`:

In [100]:
from spell.api.models import RangeSpec

h2 = client.hyper.new_random_search(
    params={
        'conv2_filter': RangeSpec(16, 128, scaling='linear', type='int'),
        'dense_layer': RangeSpec(32, 128, scaling='linear', type='int'),
        'dropout_3': RangeSpec(0.2, 0.5)
    },
    num_runs=12,
    command="python hyper/cifar10_cnn.py --epochs 25 --conv2_filter :conv2_filter: --dense_layer :dense_layer: --dropout_3 :dropout_3:",
    machine_type="K80",
    github_url="https://github.com/spellrun/examples.git",
)

Note also the presence of the `num_runs` attribute here; this controls how many jobs will be launched as part of this search. `num_jobs` is a required parameter for random and Bayesian searches only.

The individual runs associated with the hyperparameter search job are available via the `runs` attribute:

In [103]:
h2.runs[0]

Run(id=397, status='machine_requested', command='python hyper/cifar10_cnn.py --epochs 25 --conv2_filter 111 --dense_layer 53 --dropout_3 0.47596198145023644', creator=User(email='aleksey@spell.run', user_name='aleksey', full_name='Aleksey Bilogur', created_at=datetime.datetime(2020, 2, 12, 23, 28, 32, 771444, tzinfo=tzutc()), updated_at=datetime.datetime(2020, 4, 28, 0, 27, 17, 514510, tzinfo=tzutc()), last_logged_in=datetime.datetime(2020, 4, 28, 0, 27, 17, 390465, tzinfo=tzutc())), gpu='K80', git_commit_hash='c27875680c5de4a18dbff080ae6d97d1eb6bac6a', github_url='https://github.com/spellrun/examples', framework='default', created_at=datetime.datetime(2020, 5, 6, 19, 6, 7, 970882, tzinfo=tzutc()), hyper_params={'conv2_filter': 111, 'dense_layer': 53, 'dropout_3': 0.47596198145023644})

As with runs, to catch the state of the hyperparameter search object up to latest you need to run `refresh`:

In [106]:
h2.refresh()

To stop or kill the search job, use the `stop` or `kill` methods:

In [109]:
h2.kill()

You can get metrics from the hyperparameter runs by name using the `metrics_names` and `metrics` methods. The latter will return an iterator over all metrics from the search sorted by run ID.

In [6]:
h2.metric_names()

['train_loss', 'val_loss', 'val_acc']

In [12]:
metrics = h2.metrics('train_loss')
metrics

{952: <list_iterator at 0x7fa52c2b3610>,
 953: <list_iterator at 0x7fa52d093990>,
 954: <list_iterator at 0x7fa52d093690>,
 955: <list_iterator at 0x7fa52d093650>,
 956: <list_iterator at 0x7fa52d093750>,
 957: <list_iterator at 0x7fa52d093fd0>}

For more hyperparameter search methods refer to the [hyperparameter search docs](https://spell.ml/docs/hyper_searches/).

----

## models

Models are a way of organizing models you've trained on Spell. They are used by Spell model servers, but can also be exported out of Spell if, for example, you want to use the model you trained locally.

To initialize a new model (or a new version of existing model), use the `new` method. Models can be created from either runs or uploads.

In [19]:
model = client.models.new("cnn-cifar10", "runs/951")

Use `list` to get all of models in your account.

In [23]:
models = client.models.list()
len(models)

11

Each `model` object has a `model_versions` attribute, which contains a list of model versions this model has.

In [29]:
latest_model_version = model.model_versions[-1]
latest_model_version

ModelVersion(id=1, formatted_version='v1', creator=User(email='aleksey@spell.ml', user_name='aleksey', full_name='Aleksey Bilogur', created_at=datetime.datetime(2020, 2, 12, 23, 28, 32, 771444, tzinfo=tzutc()), updated_at=datetime.datetime(2021, 3, 28, 17, 13, 31, 38826, tzinfo=tzutc()), is_admin=True, last_logged_in=datetime.datetime(2021, 3, 28, 17, 13, 30, 667623, tzinfo=tzutc())), created_at=datetime.datetime(2021, 4, 22, 18, 48, 26, 792185, tzinfo=tzutc()), resource='runs/951', files=[<spell.api.models.ModelFileSpec object at 0x7fa52d3372d0>])

You can download the contents of the model version to local disc using the `download` method:

In [30]:
latest_model_version.download()

Extracting checkpoints/epoch_10.pth...
Extracting checkpoints/epoch_15.pth...
Extracting checkpoints/epoch_20.pth...
Extracting checkpoints/epoch_5.pth...
Done!


For more model methods refer to the [model docs](https://spell.ml/docs/pythonModels/).

----

## model servers

Model servers are models served as an API endpoint on a Kubernetes cluster running on Spell. You can create a new one using `serve`, list existing ones using `list`, and get an existing one using `get`.

In [34]:
client.servers.serve?

Signature: client.servers.serve(model, entrypoint, github_url, **kwargs)
Docstring:
Create a new model server using a model.

Parameters:
    model (str): Targeted model, should be in ``MODEL:VERSION`` format
    entrypoint (str): Path to the file to be used as the model server entrypoint, e.g.
        ``serve.py`` or similar.
    github_url (str): a GitHub URL to a repository for code to include in the server.
    github_ref (str, optional): a reference to a commit, branch, or tag in the repository
        corresponding to ``github_url`` for code to include in the run
        (default: ``master``).
    commit_ref (str, optional): git commit hash to use (default: ``HEAD``).
    name (str, optional): Name of the model server. Defaults to the model name.
    node_group (str, optional): Name of the node group to serve from. Defaults to the
        default node group.
    classname (str, optional): Name of the ``Predictor`` class. Only required if more then
        one predictor exists in 

In [31]:
client.servers.serve(
    entrypoint="server/serve.py",
    model="cnn-cifar10:v1",
    github_url="https://github.com/spellml/cnn-cifar10.git",
    name="cifar10"
)

In [33]:
server = client.servers.get("cifar10")

You can use `healthcheck` to hit the model server healthcheck endpoint, or use `predict` to get a prediction from the server.

In [35]:
server.healthcheck()

<Response [200]>

In [39]:
from PIL import Image
from io import BytesIO
import requests
import base64

img = Image.open("/path/to/cat.jpg")
img.convert("RGB")
buf = BytesIO()
img.save(buf, format="JPEG")
img_str = base64.b64encode(buf.getvalue())

resp = server.predict({
    "image": img_str.decode("utf8"),
    "format": "JPEG"
})
print(resp.json())

{'class': 'Cat'}


You can access server logs (which are organized by pod ID) with `logs`:

In [47]:
server.pods

[ModelServerPod(id=1, created_at=datetime.datetime(2021, 1, 26, 22, 46, 21, 526626, tzinfo=tzutc()), ready_at=datetime.datetime(2021, 1, 26, 22, 46, 58, 2506, tzinfo=tzutc()), deleted_at=datetime.datetime(2021, 2, 11, 16, 56, 36, 564970, tzinfo=tzutc())),
 ModelServerPod(id=2, created_at=datetime.datetime(2021, 2, 11, 16, 56, 42, 470212, tzinfo=tzutc()), ready_at=datetime.datetime(2021, 2, 11, 16, 57, 27, 975144, tzinfo=tzutc()), deleted_at=datetime.datetime(2021, 3, 29, 18, 49, 16, 565075, tzinfo=tzutc())),
 ModelServerPod(id=3, created_at=datetime.datetime(2021, 4, 1, 0, 39, 53, 62144, tzinfo=tzutc()), ready_at=datetime.datetime(2021, 4, 1, 0, 40, 35, 563043, tzinfo=tzutc()))]

In [44]:
list(server.logs(1))[:5]

[ModelServerLogEntry(log='Pod created. Waiting to be assigned to a machine...'),
 ModelServerLogEntry(log='Pod assigned to machine . Initializing pod...'),
 ModelServerLogEntry(log='Downloading model files and any mount files...'),
 ModelServerLogEntry(log='Download complete. Fetching image for server...'),
 ModelServerLogEntry(log='Image fetch complete. Starting server...')]

Of course you can `start`, `stop`, and `update` servers from the Python API as well.

For more server methods refer to the [model server docs](https://dev.spell.ml/docs/pythonModelServers/).